In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib.parse

# page link

In [35]:
JumiaWebsite = "https://www.jumia.com.eg/ar/mlp-men-s-jeans/?viewType=list&page="

In [ ]:
product_img =[]
free_shpping_ability=[]
product_review =[]
product_text =[]
product_price = []
product_old_price=[]
product_scorce=[]
product_link=[]
product_link_combined=[]

# cleaning data


In [ ]:
def cleandf(product_df):
    product_df["new_price"] = product_df["new_price"].str.replace("جنيه ","")
    product_df["old_price"] = product_df["old_price"].str.replace("جنيه ","")
    product_df["product_scorce"] = product_df["product_scorce"].str.replace("المتجر الرسمي","Official store")
    product_df["free_shpping_ability"] = product_df["free_shpping_ability"].str.replace("مؤهل للشحن المجاني","free shipping")

### comine links to the domain to be used

In [ ]:
domain = 'https://www.jumia.com.eg'
def combinedLink(product_link):
    for link in product_link:
        product_link_combined.append(urllib.parse.urljoin(domain,link))



# Fitch data from JUMIA

In [ ]:

def jumiaJeans(website):
    for i in range(1,51):
        website = website+str(i)+'#catalog-listing'
        response = requests.get(website)
        print(response.status_code)
        soup = BeautifulSoup(response.content,'html.parser')
        results = soup.find_all('a',{'class' : 'core'})
        for result in results:
            try:
                product_text.append(result.find('h3',{'class':'name'}).get_text())
            except:
                product_text.append('N/A')
            try:
                product_price.append(result.find('div',{'class':'prc'}).get_text())
            except:
                product_price.append('N/A')
            try:
                product_old_price.append(result.find('div',{'class':'old'}).get_text())
            except:
                product_old_price.append('N/A')
            try:
                product_review.append(result.find('div',{'class':'rev'}).get_text())
            except:
                product_review.append('N/A')
            try:
                product_img.append(result.find('img',{'class':'img'}).get('data-src'))
            except:
                product_img.append('N/A')
            try:
                product_scorce.append(result.find('div',{'class':'tag _mall _sm'}).get_text())
            except:
                product_scorce.append('N/A')
            try:
                free_shpping_ability.append(result.find('p',{'class':'shipp'}).get_text())
            except:
                free_shpping_ability.append('N/A')
            try:
                product_link.append(result.get('href'))
            except:
                product_link.append('N/A')  
    #combine links            
    combinedLink(product_link)
    #clean data
    cleandf(product_df)

In [ ]:
jumiaJeans(str(JumiaWebsite))

In [ ]:
# save to dataframe
product_df = pd.DataFrame({'product_text':product_text,'new_price':product_price,'old_price':product_old_price,'product_review':product_review,'product_img':product_img,'product_scorce':product_scorce,'free_shpping_ability':free_shpping_ability,'product_link':product_link_combined})    
product_df

# TEST

In [ ]:
product_df.head(5)

# Save to cvs file

In [ ]:
product_df.to_csv('JumiaJeans.csv')

# connecting to database

In [ ]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ABDELKADER;'
                      'Database=Jumia;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

Insert into database

In [ ]:
insert_stat = """
INSERT INTO Jeans_Info 
VALUES (?,?,?,?,?,?,?,?)
"""

In [ ]:
for text in product_df.values:
    print(list(text))
    cursor.execute(insert_stat,list(text))
conn.commit()    

select data from database

In [ ]:
cursor.execute('SELECT * FROM Jeans_Info')
for i in cursor:
    print(i)